# Data Prep for Database

In [1]:
# import modules
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from pickle import dump
from sqlalchemy import create_engine
from config import DATABASE_URI
import datetime
%matplotlib inline

pd.set_option('display.max_colwidth', -1)

C:\Users\lenpk\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [23]:
# train dataset
train = pd.read_csv('Corona_NLP_train.csv', encoding = "ISO-8859-1")
test= pd.read_csv('Corona_NLP_test.csv', encoding = "ISO-8859-1")

In [24]:
# Combine datasets
train['status'] = 'train'
test['status'] = 'test'

In [25]:
combined_df = pd.concat([test, train]).reset_index(drop = True)

In [26]:
combined_df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,status
0,1,44953,NYC,02-03-2020,"TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as #coronavirus-fearing shoppers stock up https://t.co/Gr76pcrLWh https://t.co/ivMKMsqdT1",Extremely Negative,test
1,2,44954,"Seattle, WA",02-03-2020,"When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY",Positive,test
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and loved ones from #coronavirus. ?,Extremely Positive,test
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious shoppers stock up on food&amp;medical supplies after #healthcare worker in her 30s becomes #BigApple 1st confirmed #coronavirus patient OR a #Bloomberg staged event?\r\r\n\r\r\nhttps://t.co/IASiReGPC4\r\r\n\r\r\n#QAnon #QAnon2018 #QAnon2020 \r\r\n#Election2020 #CDC https://t.co/29isZOewxu,Negative,test
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronavirusaustralia #CoronaVirusUpdate #Covid_19 #9News #Corvid19 #7NewsMelb #dunnypapergate #Costco One week everyone buying baby milk powder the next everyone buying up toilet paper. https://t.co/ScZryVvsIh,Neutral,test


In [27]:
combined_df['Location'].fillna('Unknown', inplace = True)

In [28]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44955 entries, 0 to 44954
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       44955 non-null  int64 
 1   ScreenName     44955 non-null  int64 
 2   Location       44955 non-null  object
 3   TweetAt        44955 non-null  object
 4   OriginalTweet  44955 non-null  object
 5   Sentiment      44955 non-null  object
 6   status         44955 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.4+ MB


In [29]:
# replace the extreme values
combined_df.replace({'Extremely Positive': 'Positive', 'Extremely Negative': 'Negative'}, inplace = True)

In [30]:
combined_df.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment', 'status'],
      dtype='object')

In [31]:
df_new = combined_df[['Location', 'TweetAt', 'OriginalTweet', 'Sentiment', 'status']]

In [32]:
df_new .head()

,Location,TweetAt,OriginalTweet,Sentiment,status
0,NYC,02-03-2020,"TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as #coronavirus-fearing shoppers stock up https://t.co/Gr76pcrLWh https://t.co/ivMKMsqdT1",Negative,test
1,"Seattle, WA",02-03-2020,"When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY",Positive,test
2,Unknown,02-03-2020,Find out how you can protect yourself and loved ones from #coronavirus. ?,Positive,test
3,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious shoppers stock up on food&amp;medical supplies after #healthcare worker in her 30s becomes #BigApple 1st confirmed #coronavirus patient OR a #Bloomberg staged event?\r\r\n\r\r\nhttps://t.co/IASiReGPC4\r\r\n\r\r\n#QAnon #QAnon2018 #QAnon2020 \r\r\n#Election2020 #CDC https://t.co/29isZOewxu,Negative,test
4,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronavirusaustralia #CoronaVirusUpdate #Covid_19 #9News #Corvid19 #7NewsMelb #dunnypapergate #Costco One week everyone buying baby milk powder the next everyone buying up toilet paper. https://t.co/ScZryVvsIh,Neutral,test


In [33]:
df_new.rename(columns = {'Location': 'location', 'TweetAt': 'tweeet_date', 'OriginalTweet': 'tweet', 'Sentiment': 'sentiment', 'status': 'status'}, inplace = True)

C:\Users\lenpk\anaconda3\envs\PythonAdv\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [34]:
df_new.head()

,location,tweeet_date,tweet,sentiment,status
0,NYC,02-03-2020,"TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as #coronavirus-fearing shoppers stock up https://t.co/Gr76pcrLWh https://t.co/ivMKMsqdT1",Negative,test
1,"Seattle, WA",02-03-2020,"When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY",Positive,test
2,Unknown,02-03-2020,Find out how you can protect yourself and loved ones from #coronavirus. ?,Positive,test
3,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious shoppers stock up on food&amp;medical supplies after #healthcare worker in her 30s becomes #BigApple 1st confirmed #coronavirus patient OR a #Bloomberg staged event?\r\r\n\r\r\nhttps://t.co/IASiReGPC4\r\r\n\r\r\n#QAnon #QAnon2018 #QAnon2020 \r\r\n#Election2020 #CDC https://t.co/29isZOewxu,Negative,test
4,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronavirusaustralia #CoronaVirusUpdate #Covid_19 #9News #Corvid19 #7NewsMelb #dunnypapergate #Costco One week everyone buying baby milk powder the next everyone buying up toilet paper. https://t.co/ScZryVvsIh,Neutral,test


In [35]:
df_final = df_new[['tweeet_date', 'location', 'tweet', 'sentiment', 'status']]

In [44]:
df_final['tweeet_date'] = pd.to_datetime(df_final['tweeet_date'])

In [45]:
df_final.tail()

,tweeet_date,location,tweet,sentiment,status
44950,2020-04-14,"Wellington City, New Zealand",Airline pilots offering to stock supermarket shelves in #NZ lockdown #COVID-19 https://t.co/cz89uA0HNp,Neutral,train
44951,2020-04-14,Unknown,Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?,Negative,train
44952,2020-04-14,Unknown,"You know itÂs getting tough when @KameronWilds is rationing toilet paper #coronavirus #toiletpaper @kroger martinsville, help us out!!",Positive,train
44953,2020-04-14,Unknown,Is it wrong that the smell of hand sanitizer is starting to turn me on?\r\r\n\r\r\n#coronavirus #COVID19 #coronavirus,Neutral,train
44954,2020-04-14,i love you so much || he/him,@TartiiCat Well new/used Rift S are going for $700.00 on Amazon rn although the normal market price is usually $400.00 . Prices are really crazy right now for vr headsets since HL Alex was announced and it's only been worse with COVID-19. Up to you whethe,Negative,train


In [2]:
# Create connection
engine = create_engine(DATABASE_URI)
conn = engine.connect()

In [3]:
# Check tables if loaded
engine.table_names()

['tweets_sentiment']

In [49]:
# Load df_final in database
df_final.to_sql(name = 'tweets_sentiment', con = engine, if_exists = 'append', index = False)

In [18]:
df = pd.read_sql_query('SELECT * FROM tweets_sentiment', con = engine)

In [21]:
test_dt = df.loc[df['tweeet_date'] == datetime.date(2020, 11, 3)]
test_dt 

,id,tweeet_date,location,tweet,sentiment,status
51,1147,2020-11-03,"Dallas, TX","I was quoted in this piece - ""Dos and don'ts of stockpiling: 5 must-have items"": https://t.co/U8ECcFeOzE by @eviefordham https://t.co/oFqrPP4r9V",Neutral,test
112,1071,2020-11-03,United States,"@realDonaldTrump What can you do about #COVID2019? \r\r\n\r\r\n- donÂt go out sick\r\r\n- wash hands\r\r\n- call ahead, wear mask to doctor\r\r\n- stock up food, medicines\r\r\n- work from home\r\r\n- keep social distances\r\r\n- hand sanitizer after surfaces\r\r\n- avoid touching",Negative,test
113,1072,2020-11-03,England.,As I appear to be absolutely no good at stock piling/panic buying food goods and toilet rolls for the #coronavirus I'm moving on to something I'm good at buying ?\r\r\n\r\r\nTin whistles and Apple Watch bands.\r\r\n\r\r\nBetter than toilet rolls ? https://t.co/UdI4PYuLDh,Positive,test
114,1073,2020-11-03,United States,"Pretty good chance that your waiter/waitress, grocery/department store clerk, or Uber/Taxi driver, doesn't have health insurance or paid sick leave. \r\r\n\r\r\nAlso a pretty good chance that these people come in contact with #COVID19 #coronavirus \r\r\n\r\r\nStill think Bernie's radical? Good luck.",Positive,test
115,1074,2020-11-03,"Highland, MI","I donÂt understand the run on toilet paper. \r\r\n\r\r\nYou would think that youÂd stock up on food first. Without food, you wonÂt need the toilet paper. #coronavirus",Negative,test
...,...,...,...,...,...,...
274,1231,2020-11-03,"Chicago, IL",Wondering if I should stock up on food and home goods. ? what are yÂall doing? \r\r\n#coronavirus,Neutral,test
275,1232,2020-11-03,London,The clowns are playing games with #COVID2019 and peoples lives.\r\r\n\r\r\nLarge parts of the community are concerned for vunerable family &amp; friends. The clowns are monitoring consumer spending.,Positive,test
276,1233,2020-11-03,Fashion Capital,@ElieTahari #samplesale is #postponed - expect other #sales to go #online or skip a #season as #coronavirus #pandemic grows #shopping #nyc https://t.co/bXxD4n8oI8,Neutral,test
277,1234,2020-11-03,Unknown,"#Jamaica, sorry to burst your bubble but #Lysol cannot manage this strain of #coronavirus. The one on the Lysol tin is an older version of #coronavirus. #COVID19 was built to defeat #Lysol. Stop wasting money on #Lysol &amp; buy soap &amp; alcohol &amp; stock up on food &amp; medication.",Negative,test


In [35]:
df_sent = pd.read_sql_query("""SELECT tweeet_date, sentiment FROM tweets_sentiment;""", conn)
dfMain = df_sent.loc[df_sent['sentiment'] == 'Positive']
dfLabel = dfMain.groupby('tweeet_date').count().reset_index()
dfLabel.head()

,tweeet_date,sentiment
0,2020-01-04,259
1,2020-02-03,2
2,2020-02-04,392
3,2020-03-03,1
4,2020-03-04,345


In [37]:
dfLabel.columns

Index(['tweeet_date', 'sentiment'], dtype='object')

In [16]:
df_dates['tweeet_date'].unique()

array([datetime.date(2020, 2, 3), datetime.date(2020, 3, 3),
       datetime.date(2020, 4, 3), datetime.date(2020, 5, 3),
       datetime.date(2020, 6, 3), datetime.date(2020, 7, 3),
       datetime.date(2020, 3, 13), datetime.date(2020, 8, 3),
       datetime.date(2020, 9, 3), datetime.date(2020, 11, 3),
       datetime.date(2020, 10, 3), datetime.date(2020, 3, 22),
       datetime.date(2020, 12, 3), datetime.date(2020, 3, 24),
       datetime.date(2020, 3, 31), datetime.date(2020, 2, 4),
       datetime.date(2020, 3, 29), datetime.date(2020, 3, 15),
       datetime.date(2020, 3, 17), datetime.date(2020, 4, 16),
       datetime.date(2020, 4, 17), datetime.date(2020, 3, 18),
       datetime.date(2020, 3, 19), datetime.date(2020, 3, 14),
       datetime.date(2020, 4, 18), datetime.date(2020, 3, 20),
       datetime.date(2020, 4, 19), datetime.date(2020, 4, 20),
       datetime.date(2020, 3, 23), datetime.date(2020, 3, 16),
       datetime.date(2020, 3, 25), datetime.date(2020, 3, 27),
 

In [45]:
df_sent = pd.read_sql_query("""SELECT tweeet_date, tweet, sentiment FROM tweets_sentiment;""", conn)
    
df_sent['dates'] = pd.to_datetime(df_sent['tweeet_date'], errors='coerce')
df_sent['month'] = df_sent['dates'].dt.strftime('%b')
dfMain = df_sent.loc[df_sent['month'] == 'May']
dfLabel = dfMain.groupby('sentiment')['tweet'].count().reset_index()

In [46]:
dfLabel

,sentiment,tweet
0,Negative,383
1,Neutral,236
2,Positive,518


In [47]:
df_new = pd.read_sql_query("""SELECT * FROM tweets_sentiment;""", conn)

In [49]:
df_new.tail()

,id,tweeet_date,location,tweet,sentiment,status
214553,215514,2020-04-30,Unknown,"Who do you want to be during #covid19 ? How are you showing up for yourself, your family, your patients? #womenshealth #obgyn #mindfulness #parenting https://t.co/uPt8ftzOKi",Positive,predicted
214554,215515,2020-04-30,Unknown,Epic cancels 2020 Fortnite World Cup https://t.co/dmtupRNqF0 #fortniteworldcup #coronavirus #videogames #epicgames #fortnite #covid19 #games,Positive,predicted
214555,215516,2020-04-30,Unknown,"@APompliano @zerohedge Considering the most cases of #coronavirus are in #NewYork, #Massachusetts &amp; #NewJersey in America, and in #Canada #Quebec &amp; #Ontario, it would make perfect sense given the distance between these places &amp; #Maryland Fort Detrick. https://t.co/B5rkrioZNM https://t.co/PS4ZXoqT33",Positive,predicted
214556,215517,2020-04-30,Unknown,Just imagine... #covid_19 #stayhomestaysafe https://t.co/4t4A5lq4cf,Neutral,predicted
214557,215518,2020-04-30,Unknown,Startling context. AIDS deaths v #COVID19 https://t.co/2xazyKTxex,Neutral,predicted


In [51]:
conn.close()